In [1]:
# --- Instalación de librerías adicionales ---
!pip install polyline

# --- Importaciones ---
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import joblib
import requests
import os
from datetime import datetime
import warnings
import time
import polyline

# --- Importar el gestor de secretos de Kaggle ---
from kaggle_secrets import UserSecretsClient

# --- Configuración Inicial ---
warnings.simplefilter(action='ignore')
pd.set_option('display.max_rows', 100)

print("Librerías importadas y configuración inicial completada.")

Librerías importadas y configuración inicial completada.


In [2]:
# --- Cargar la clave de API desde Kaggle Secrets ---
user_secrets = UserSecretsClient()
API_KEY_GOOGLE = user_secrets.get_secret("GOOGLE_API_KEY")

# --- Variables de Configuración ---
# Ruta representativa en Camp de Túria (Llíria a Bétera)
ORIGIN_COORDS = "39.6253,-0.5961"
DESTINATION_COORDS = "39.5925,-0.4619"

# Rutas de archivos dentro del entorno de Kaggle
KAGGLE_WORKING_DIR = "/kaggle/working/"
HISTORICO_CSV_PATH = os.path.join(KAGGLE_WORKING_DIR, 'historico_trafico_avanzado.csv')
MODELO_PATH = os.path.join(KAGGLE_WORKING_DIR, 'modelo_trafico_avanzado.pkl')

print(f"La API Key ha sido cargada de forma segura.")
print(f"El archivo de datos se guardará en: {HISTORICO_CSV_PATH}")
print(f"El modelo se guardará en: {MODELO_PATH}")

La API Key ha sido cargada de forma segura.
El archivo de datos se guardará en: /kaggle/working/historico_trafico_avanzado.csv
El modelo se guardará en: /kaggle/working/modelo_trafico_avanzado.pkl


In [3]:
# ==============================================================================
# PARTE A: RECOLECTOR DE DATOS AVANZADO (VERSIÓN MODIFICADA)
# ==============================================================================

def obtener_datos_de_ruta_avanzados():
    """
    Orquesta las llamadas a las APIs de Google para obtener datos de ruta,
    límites de velocidad y matriz de distancia.
    """
    print("--- PARTE A: RECOLECTOR DE DATOS AVANZADO ---")
    if API_KEY_GOOGLE == "AQUI_VA_TU_API_KEY" or not API_KEY_GOOGLE:
        print("ADVERTENCIA: No se ha configurado una API Key de Google. No se pueden obtener datos reales.")
        return None

    try:
        # 1. Obtener la ruta principal y la polilínea con Routes API
        print("1/3: Obteniendo ruta y polilínea de Routes API...")
        routes_data = llamar_routes_api(ORIGIN_COORDS, DESTINATION_COORDS)
        if not routes_data or 'routes' not in routes_data or not routes_data['routes']:
            print("Error: Routes API no devolvió una ruta válida.")
            return None
        
        route = routes_data['routes'][0]
        # encoded_polyline = route['polyline']['encodedPolyline'] # Ya no la necesitamos
        duracion_con_trafico_seg = int(route['duration'].replace('s', ''))
        distancia_metros = route['distanceMeters']

        # 2. OMITIDO: Obtener límites de velocidad con Roads API
        # Esta función requiere una cuenta de facturación activa en Google Cloud.
        # Para que el script funcione, la omitimos y usamos un valor fijo.
        print("2/3: OMITIENDO obtención de límites de velocidad (requiere facturación).")
        velocidad_limite_promedio_kmh = 90.0 # Usamos un valor fijo como placeholder

        # 3. Obtener duración sin tráfico con Distance Matrix API
        print("3/3: Obteniendo datos de Distance Matrix API...")
        matrix_data = llamar_distance_matrix_api(ORIGIN_COORDS, DESTINATION_COORDS)
        if not matrix_data:
            return None
        duracion_sin_trafico_seg = matrix_data['duration']['value']

        # Calcular nuevas características
        factor_congestion = duracion_con_trafico_seg / duracion_sin_trafico_seg if duracion_sin_trafico_seg > 0 else 1.0

        nuevo_registro = {
            'timestamp': datetime.now(),
            'duracion_viaje_seg': duracion_con_trafico_seg,
            'distancia_metros': distancia_metros,
            'velocidad_limite_promedio_kmh': velocidad_limite_promedio_kmh,
            'factor_congestion': round(factor_congestion, 2)
        }
        
        # Guardar en el CSV
        df_nuevo = pd.DataFrame([nuevo_registro])
        df_nuevo.to_csv(HISTORICO_CSV_PATH, mode='a', header=not os.path.exists(HISTORICO_CSV_PATH), index=False)
        print(f"Éxito. Nuevo registro guardado: {nuevo_registro}")
        return nuevo_registro

    except requests.exceptions.RequestException as e:
        print(f"Error de red al llamar a las APIs de Google: {e}")
        return None
    except (KeyError, IndexError) as e:
        print(f"Error de formato en la respuesta de la API: {e}")
        return None
    except Exception as e:
        print(f"Un error inesperado ocurrió: {e}")
        return None


def llamar_routes_api(origin, destination):
    url = "https://routes.googleapis.com/directions/v2:computeRoutes"
    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': API_KEY_GOOGLE,
        'X-Goog-FieldMask': 'routes.duration,routes.distanceMeters,routes.polyline'
    }
    payload = {
        "origin": {"location": {"latLng": {"latitude": float(origin.split(',')[0]), "longitude": float(origin.split(',')[1])}}},
        "destination": {"location": {"latLng": {"latitude": float(destination.split(',')[0]), "longitude": float(destination.split(',')[1])}}},
        "travelMode": "DRIVE",
        "routingPreference": "TRAFFIC_AWARE",
    }
    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status()
    return response.json()

# La función obtener_limite_velocidad_promedio ya no es necesaria, pero la dejamos por si la activas en el futuro
def obtener_limite_velocidad_promedio(encoded_polyline_str):
    """Decodifica una polilínea y obtiene el límite de velocidad promedio de la Roads API."""
    path = polyline.decode(encoded_polyline_str)
    if len(path) > 100:
        indices = np.linspace(0, len(path) - 1, 100, dtype=int)
        path = [path[i] for i in indices]

    path_str = "|".join([f"{lat},{lon}" for lat, lon in path])
    url = f"https://roads.googleapis.com/v1/speedLimits?path={path_str}&key={API_KEY_GOOGLE}"
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    if 'speedLimits' not in data or not data['speedLimits']: return 0.0
    limites = [sl['speedLimit'] for sl in data['speedLimits']]
    return round(np.mean(limites), 2) if limites else 0.0


def llamar_distance_matrix_api(origin, destination):
    url = (f"https://maps.googleapis.com/maps/api/distancematrix/json?"
           f"origins={origin}&destinations={destination}&mode=driving&key={API_KEY_GOOGLE}")
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    if data['status'] == 'OK' and data['rows'][0]['elements'][0]['status'] == 'OK':
        return data['rows'][0]['elements'][0]
    else:
        print(f"Error en Distance Matrix API: {data.get('error_message', data['status'])}")
        return None

print("Funciones del recolector de datos (Parte A) definidas con la omisión de 'speedLimits'.")

Funciones del recolector de datos (Parte A) definidas con la omisión de 'speedLimits'.


In [4]:
# ==============================================================================
# PARTE B: ENTRENADOR Y PRONOSTICADOR DE ML
# ==============================================================================

def entrenar_y_predecir():
    """Carga datos históricos enriquecidos, entrena un modelo y predice el tráfico."""
    print("\n--- PARTE B: ENTRENADOR Y PRONOSTICADOR DE ML (AVANZADO) ---")
    
    # --- PASO 1: Cargar datos ---
    if not os.path.exists(HISTORICO_CSV_PATH):
        print(f"Error: El archivo '{HISTORICO_CSV_PATH}' no existe. Ejecute el recolector primero.")
        return
        
    df = pd.read_csv(HISTORICO_CSV_PATH, parse_dates=['timestamp'])
    
    if len(df) < 20: # Reducido para permitir entrenar antes
        print(f"ADVERTENCIA: Hay muy pocos datos ({len(df)} registros). La predicción no será precisa.")

    # --- PASO 2: Ingeniería de Características ---
    df['hora'] = df['timestamp'].dt.hour
    df['dia_semana'] = df['timestamp'].dt.dayofweek
    df['es_finde'] = (df['dia_semana'] >= 5).astype(int)
    
    features = ['hora', 'dia_semana', 'es_finde', 'distancia_metros', 'velocidad_limite_promedio_kmh']
    target = 'duracion_viaje_seg'
    
    X = df[features]
    y = df[target]

    if X.empty or y.empty:
        print("No hay suficientes datos para entrenar el modelo.")
        return

    # Solo se puede hacer split si hay más de 1 muestra
    if len(df) > 1:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    else:
        X_train, y_train = X, y
        X_test, y_test = pd.DataFrame(), pd.Series()


    # --- PASO 3: Entrenar el modelo ---
    print("PASO 3: Entrenando el modelo RandomForestRegressor...")
    model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1, min_samples_leaf=2)
    model.fit(X_train, y_train)
    joblib.dump(model, MODELO_PATH)
    print(f"Modelo entrenado y guardado como '{MODELO_PATH}'.\n")

    # --- PASO 4: Evaluar el modelo ---
    if not y_test.empty:
        print("PASO 4: Evaluando el rendimiento del modelo...")
        predictions = model.predict(X_test)
        mae = mean_absolute_error(y_test, predictions)
        r2 = r2_score(y_test, predictions)
        print(f"  - Error Medio Absoluto (MAE): {mae:.2f} segundos (~{int(mae/60)} min)")
        print(f"  - Coeficiente R²: {r2:.2f}\n")
    
    # --- PASO 5: Pronosticar las próximas 24 horas ---
    print("PASO 5: Pronosticando las próximas 24 horas...")
    future_timestamps = pd.date_range(start=datetime.now(), periods=24, freq='h')
    df_futuro = pd.DataFrame({'timestamp': future_timestamps})
    df_futuro['hora'] = df_futuro['timestamp'].dt.hour
    df_futuro['dia_semana'] = df_futuro['timestamp'].dt.dayofweek
    df_futuro['es_finde'] = (df_futuro['dia_semana'] >= 5).astype(int)
    
    # Usar los valores promedio/constantes de las otras características del dataframe histórico
    df_futuro['distancia_metros'] = df['distancia_metros'].iloc[0] if not df.empty else 0
    df_futuro['velocidad_limite_promedio_kmh'] = df['velocidad_limite_promedio_kmh'].mean() if not df.empty else 0

    X_futuro = df_futuro[features]
    pronostico = model.predict(X_futuro)
    df_futuro['duracion_predicha_seg'] = pronostico.astype(int)
    df_futuro['duracion_predicha_min'] = (df_futuro['duracion_predicha_seg'] / 60).round(1)

    print("\n--- Pronóstico de Duración de Viaje para 'Camp de Túria' (Llíria -> Bétera) ---")
    print(df_futuro[['timestamp', 'duracion_predicha_min']].to_string(index=False))

print("Función del entrenador y pronosticador (Parte B) definida.")

Función del entrenador y pronosticador (Parte B) definida.


In [5]:
# --- INICIO DE LA EJECUCIÓN ---

# 1. Ejecutar el recolector para obtener el dato más reciente
print("Ejecutando el recolector de datos...")
obtener_datos_de_ruta_avanzados()

# 2. Entrenar con todos los datos disponibles y predecir el futuro
print("\nEjecutando el entrenador y pronosticador...")
entrenar_y_predecir()

Ejecutando el recolector de datos...
--- PARTE A: RECOLECTOR DE DATOS AVANZADO ---
1/3: Obteniendo ruta y polilínea de Routes API...
2/3: OMITIENDO obtención de límites de velocidad (requiere facturación).
3/3: Obteniendo datos de Distance Matrix API...
Éxito. Nuevo registro guardado: {'timestamp': datetime.datetime(2025, 6, 30, 15, 47, 57, 921538), 'duracion_viaje_seg': 1187, 'distancia_metros': 17268, 'velocidad_limite_promedio_kmh': 90.0, 'factor_congestion': 0.97}

Ejecutando el entrenador y pronosticador...

--- PARTE B: ENTRENADOR Y PRONOSTICADOR DE ML (AVANZADO) ---
ADVERTENCIA: Hay muy pocos datos (1 registros). La predicción no será precisa.
PASO 3: Entrenando el modelo RandomForestRegressor...
Modelo entrenado y guardado como '/kaggle/working/modelo_trafico_avanzado.pkl'.

PASO 5: Pronosticando las próximas 24 horas...

--- Pronóstico de Duración de Viaje para 'Camp de Túria' (Llíria -> Bétera) ---
                 timestamp  duracion_predicha_min
2025-06-30 15:47:58.174636  